# Qulacs Python Advanced Guide

This chapter is for those who are familiar with the terminology of quantum information and want fine tuning for numerical calculations. For more information on terminology, please see the textbook **Quantum Computation and Quantum Information** by M.A. Nielsen et al..

## Quantum States
This class allocates and manages $2^n$ complex arrays on the CPU/GPU with the precision of complex128.

### Create and Destroy
Necessary memory is secured at the time of instance creation and released when the instance is destroyed, but you can explicitly destroy it to release the memory with `del`.

In [1]:
from qulacs import QuantumState
n = 2
state = QuantumState(n)
print(state)
del state


 *** Quantum State ***
 * Qubit Count : 2
 * Dimension   : 4
 * State vector : 
(1,0)
(0,0)
(0,0)
(0,0)



### Transform between quantum state and numpy array
Transformation between quantum state and numpy array can be realized by `get_vector` or `load` function. In principle, it is not checked whether the norm is saved.
While `get_vector` returns all the element as an array, `get_amplitude` can be used to quickly get a single element.

In [3]:
from qulacs import QuantumState

state = QuantumState(2)
vec = state.get_vector()
print(vec)
state.load([0,1,2,3])
print(state.get_vector())
print(state.get_amplitude(2))


[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 1.+0.j 2.+0.j 3.+0.j]
(2+0j)


### Copy quantum states
Quantum states can generate new instances of the same state by `copy`. By giving a quantum state to the load function, it is possible to copy a quantum vector of another quantum state without allocating a new area in the existing quantum state. This allows you to reuse the already allocated space. You can use the `allocate_buffer` function if you want to allocate a state vector of the same size as the quantum state you already have and without copying the state.

In [4]:
from qulacs import QuantumState

initial_state = QuantumState(3)
buffer = initial_state.allocate_buffer()
for ind in range(10):
    buffer.load(initial_state)
    # some computation and get results


### Store quantum state
Quantum states can be converted to JSON string.
You can restore a quantum state by loading JSON.

In [5]:
from qulacs import QuantumState
from qulacs import state

o_state = QuantumState(2)
o_state.set_Haar_random_state()
print("original:", o_state.get_vector())
state_json = o_state.to_json()

r_state = state.from_json(state_json)
print("restored:", r_state.get_vector())


original: [-0.53409555-0.11980945j  0.23272193+0.42256166j -0.65314356-0.07012461j
 -0.02987127+0.18778582j]
restored: [-0.53409555-0.11980945j  0.23272193+0.42256166j -0.65314356-0.07012461j
 -0.02987127+0.18778582j]


Quantum states can also be saved to files in pickle format.

In [6]:
from qulacs import QuantumState
import pickle

state = QuantumState(2)

# store
with open('state.pickle', 'wb') as f:
    pickle.dump(state, f)

# load
with open('state.pickle', 'rb') as f:
    state = pickle.load(f)


### Initialization of quantum state
The following is an function initializing a quantum state to a specific state.

In [7]:
from qulacs import QuantumState

n = 3
state = QuantumState(n)
# Initialize as |0> state
state.set_zero_state()
print(state.get_vector())

# Initialize the specified value to the calculation base in binary notation
state.set_computational_basis(0b101)
print(state.get_vector())

# Initialize to random pure state with Haar measure using argument value as seed
# If no value is specified, the time function is used as a seed. Pseudo random number uses xorshift.
state.set_Haar_random_state(0)
print(state.get_vector())


[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[-0.08457288-0.14143014j -0.13386446+0.11328032j -0.22521966-0.16546737j
 -0.15105932+0.48125064j -0.45087363+0.17271267j -0.05855838+0.32498025j
  0.35972119+0.02643361j -0.10103482-0.35651694j]


### Check quantum state
The following example is a list of functions to check quantum state information without changing the quantum state.

In [8]:
from qulacs import QuantumState

n = 5
state = QuantumState(n)
state.set_Haar_random_state(0)

# Get quantum bit numbers
qubit_count = state.get_qubit_count()
print("qubit_count", qubit_count)

# Get the probability that the specified qubit will be measured as 0
prob = state.get_zero_probability(1)
print("zero_prob_1", prob)

# Get arbitrary marginal probabilities
# Argument is an array of the same length as the number of qubits
# Specify 0,1,2. 0,1 is the probability of the subscript measured at that value
# 2 means that bit is peripheralized.
# For example, calculation of the probability that the third is measured as 0 and the 0th is measured as 1:
prob = state.get_marginal_probability([1,2,2,0,2])
print("marginal_prob", prob)

# Get the entropy of the probability distribution when measured on the Z basis
ent = state.get_entropy()
print("entropy", ent)

# Get squared norm (<a|a>)
# Because the operation may not be Trace preserving, the norm of state does not necessarily to be 1.
sq_norm = state.get_squared_norm()
print("sqaured_norm", sq_norm)

# Measure and sample all the qubits on Z-basis as many times as given by the argument. 
# Returns a list of integers converted from the resulting binaries.
samples = state.sampling(10)
print("sampling", samples)

# You can supply a random seed as second argument.
# If the same seed is given, always returns the same sampling result.
samples_with_seed = state.sampling(10, 314)
print("sampling (with seed)", samples_with_seed)

# Get a character string indicating whether the state vector is on CPU or GPU
dev_type = state.get_device_name()
print("device", dev_type)


qubit_count 5
zero_prob_1 0.4601075596424598
marginal_prob 0.20030608663813237
entropy 3.1082736424124735
sqaured_norm 0.9999999999999999
sampling [11, 18, 12, 29, 21, 13, 28, 24, 10, 22]
sampling (with seed) [23, 18, 28, 14, 17, 30, 9, 17, 16, 10]
device cpu


### Deformation of quantum state
The following functions modify a quantum state.

In [9]:
from qulacs import QuantumState
state = QuantumState(2)
state.set_computational_basis(0)
buffer = QuantumState(2)
buffer.set_computational_basis(2)
print("state" , state.get_vector())
print("buffer", buffer.get_vector())

# Sum of quantum state (state <- state+buffer)
# Add the buffer state to the state to create a superposition state.
# The norm after the operation generally is not 1.
state.add_state(buffer)
print("added", state.get_vector())

# Product of quantum state and complex number
# Multiplies all elements by the complex number of the argument. 
# The norm after operation generally is not 1.
coef = 0.5 + 0.1j
state.multiply_coef(coef)
print("mul_coef", state.get_vector())

# Pruduct of a quantum state and complex numbers specified by an index of each element
# Multiplies the amplitude of |00> by `coef_func(0)`, the amplitude of |01> by `coef_func(1)`.
# The norm after operation generally is not 1.
def coef_func(i: int) -> complex:
    assert 0 <= i < 2**2
    return 1j**i
state.multiply_elementwise_function(coef_func)
print("mul_elementwise_func", state.get_vector())

# Normalize quantum states
# Provide the current squared norm as an argument.
squared_norm = state.get_squared_norm()
print("sq_norm", squared_norm)
state.normalize(squared_norm)
print("normalized", state.get_vector())
print("sq_norm", state.get_squared_norm())


state [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
buffer [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
added [1.+0.j 0.+0.j 1.+0.j 0.+0.j]
mul_coef [0.5+0.1j 0. +0.j  0.5+0.1j 0. +0.j ]
mul_elementwise_func [ 0.5+0.1j  0. +0.j  -0.5-0.1j  0. -0.j ]
sq_norm 0.52
normalized [ 0.69337525+0.13867505j  0.        +0.j         -0.69337525-0.13867505j
  0.        -0.j        ]
sq_norm 0.9999999999999998


### Operation on classic registers
Quantum states have classical registers as integer arrays with variable length. The classical register is used to write the result of the Instrument operation or to describe a gate that executes conditions as the result of the classical register. The value of a classic register that has not yet been written is 0. The classical register is copied at the same time when the quantum state is copied by the `copy` and `load` functions.

In [10]:
from qulacs import QuantumState
state = QuantumState(3)
position = 0
# Write the value to `position`-th register
state.set_classical_value(position, 20)
# Get the value of the `position`-th register
obtained = state.get_classical_value(position)
print(obtained)


20


### Calculation between quantum states
The inner product and tensor product between quantum states can be obtained by `inner_product` and `tensor_product` respectively.

In [11]:
from qulacs import QuantumState
from qulacs.state import inner_product, tensor_product

n = 5
state_bra = QuantumState(n)
state_ket = QuantumState(n)
state_bra.set_Haar_random_state()
state_ket.set_computational_basis(0)

# Calculation of inner product
value = inner_product(state_bra, state_ket)
print(value)

n1 = 1
state_ket1 = QuantumState(n1)
state_ket1.set_computational_basis(1)
n2 = 2
state_ket2 = QuantumState(n2)
state_ket2.set_computational_basis(2)

# Calculation of tensor product
tensor_product_state = tensor_product(state_ket1, state_ket2)
print(tensor_product_state.get_vector())


(-0.056292589186392766-0.06355316981838662j)
[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]


### Swap and delete qubits
You can swap indices of a qubit with `permutate_qubit()`.
You can get a projection onto a specified qubit with `drop_qubit()`.

In [12]:
from qulacs import QuantumState
from qulacs.state import permutate_qubit, drop_qubit

n = 3
state = QuantumState(n)
state.set_Haar_random_state()
print("original:", state.get_vector())
# new qubit 0 is old qubit 1
# new qubit 1 is old qubit 2, 
# new qubit 2 is old qubit 0, 
permutate = permutate_qubit(state, [1, 2, 0])
print("permutate:", permutate.get_vector())
print()

n = 3
state = QuantumState(n)
state.set_Haar_random_state()
print("original:", state.get_vector())
state0 = drop_qubit(state, [1], [0])
print("projection to 0:", state0.get_vector()) # projection: qubit 1 is 0
state1 = drop_qubit(state, [1], [1])
print("projection to 1:", state1.get_vector()) # projection: qubit 1 is 1


original: [ 0.04935285-0.33525621j -0.40848741-0.16861513j  0.0877062 +0.3016868j
  0.43151866+0.12561444j  0.05137289-0.07313601j  0.10155306-0.05927549j
  0.12474423-0.07303469j  0.23710293+0.53875064j]
permutate: [ 0.04935285-0.33525621j  0.0877062 +0.3016868j   0.05137289-0.07313601j
  0.12474423-0.07303469j -0.40848741-0.16861513j  0.43151866+0.12561444j
  0.10155306-0.05927549j  0.23710293+0.53875064j]

original: [-0.13129252+0.41425428j -0.06117213+0.11127952j -0.30889329-0.26279468j
 -0.14177694+0.05919262j  0.03556784-0.19266318j -0.24280479-0.45063361j
  0.383783  -0.0233139j   0.34842192+0.19315843j]
projection to 0: [-0.13129252+0.41425428j -0.06117213+0.11127952j  0.03556784-0.19266318j
 -0.24280479-0.45063361j]
projection to 1: [-0.30889329-0.26279468j -0.14177694+0.05919262j  0.383783  -0.0233139j
  0.34842192+0.19315843j]


### Calculate partial trace
With `partial_trace()`, you can obtain a partial trace of a given qubit of a given quantum state as a density matrix.
The indices of the converted qubits are reassigned based on the order of the qubits before conversion.

In [13]:
from qulacs import QuantumState, DensityMatrix
from qulacs.gate import H, X
from qulacs.state import partial_trace

state = QuantumState(3)
state.set_computational_basis(0)
H(0).update_quantum_state(state)
X(1).update_quantum_state(state)
print(state.get_vector())

trace = partial_trace(state, [1])
print(trace.get_matrix())

dm_state = DensityMatrix(3)
dm_state.set_computational_basis(0)
H(0).update_quantum_state(dm_state)
X(1).update_quantum_state(dm_state)
print(dm_state.get_matrix())

trace = partial_trace(dm_state, [1])
print(trace.get_matrix())


[0.        +0.j 0.        +0.j 0.70710678+0.j 0.70710678+0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
[[0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]]
[[0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0.5+0.j 0.5+0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j 0. +0.j]]
[[0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0.5+0.j 0.5+0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]]


### Calculation using GPU
When Qulacs is installed from qulacs-gpu package, `QuantumStateGpu` is available. Except the different class name, the usage is the same as `QuantumState`.

```python
from qulacs import QuantumStateGpu
state = QuantumStateGpu(2)
print(state)
# print(state.get_device_name())
# gpu
```

Though the usage is the same as `QuantumState`, there are two aspects to keep in mind:

* The `get_vector` function takes a long time because it requires copying between the GPU and CPU. This function should be avoided whenever possible.
* `inner_product` between CPU / GPU states cannot be calculated. It is possible to `load` a state vector between the GPU and CPU state vectors, but it is time consuming and should be avoided.

## DensityMatrix

`DensityMatrix` is a class to hold a quantum state as a density matrix. While `StateVector` can hold a pure state, `DensityMatrix` can also hold a mixed state, or probabilistic mixture of multiple states.
Use the density matrix $\sum_i p_i\ket{\psi_i}\bra{\psi_i}$ when each state is $\ket{\psi_i}$ with probability $p_i$.
This may seem redundant since there are no multi-state composite states in this chapter, but it will be useful when using `Probabilistic` gates, etc., described below.
Basically, `DensityMatrix` can be operated in the same way as `QuantumState`.

### Generate and delete
The necessary memory is allocated when the instance is created. The memory is released when Python interpreter destroys the instance, but can be released with `del` if you want to explicitly destroy it to free memory.

In [3]:
from qulacs import DensityMatrix

n = 2
state = DensityMatrix(n)
print(state)
del state


 *** Density Matrix ***
 * Qubit Count : 2
 * Dimension   : 4
 * Density matrix : 
(1,0) (0,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (0,0)



### Conversion between quantum states and numpy array
You can convert quantum states and numpy array mutually with `get_matrix` and `load`.
If the array is 1-dimensional, it is converted from a state vector to a density matrix, and if it is 2-dimensional, it is read as a density matrix.
Basically, whether the norm is conserved or not is not checked.

In [15]:
from qulacs import DensityMatrix

state = DensityMatrix(2)
mat = state.get_matrix()
print(mat)
state.load([0,1,2,3])
print(state.get_matrix())
state.load([[0,1,2,3], [1,2,3,4], [2,3,4,5], [3,4,5,6]])
print(state.get_matrix())


[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 2.+0.j 3.+0.j]
 [0.+0.j 2.+0.j 4.+0.j 6.+0.j]
 [0.+0.j 3.+0.j 6.+0.j 9.+0.j]]
[[0.+0.j 1.+0.j 2.+0.j 3.+0.j]
 [1.+0.j 2.+0.j 3.+0.j 4.+0.j]
 [2.+0.j 3.+0.j 4.+0.j 5.+0.j]
 [3.+0.j 4.+0.j 5.+0.j 6.+0.j]]


### Copy between quantum states
A quantum state can be instantiated by `copy` to create a new instance of itself.
Also, by giving a quantum state to the `load` function, you can copy the quantum vector or density matrix of another quantum state without allocating new space in the existing quantum state.
This allows you to reuse the space you have already allocated. If you want to allocate a density matrix of the same size as the quantum state you already have, but do not need to copy the state, you can use the `allocate_buffer` function.

In [16]:
from qulacs import QuantumState, DensityMatrix

initial_state = DensityMatrix(3)
copied_state = initial_state.copy()
buffer = initial_state.allocate_buffer()
buffer.load(initial_state)
state_vector = QuantumState(3)
buffer.load(state_vector)


### Store quantum states
`DensityMatrix` can also be converted to/from JSON string

In [17]:
from qulacs import DensityMatrix
from qulacs import state

o_state = DensityMatrix(2)
o_state.set_Haar_random_state()
print("original:", o_state.get_matrix())
state_json = o_state.to_json()

r_state = state.from_json(state_json)
print("restored:", r_state.get_matrix())


original: [[ 0.69553616+0.j         -0.30796432+0.24210111j -0.00753329-0.048494j
  -0.16911174-0.16523753j]
 [-0.30796432-0.24210111j  0.22062831+0.j         -0.01354418+0.02409399j
   0.01736242+0.13202679j]
 [-0.00753329+0.048494j   -0.01354418-0.02409399j  0.00346268+0.j
   0.01335228-0.01000109j]
 [-0.16911174+0.16523753j  0.01736242-0.13202679j  0.01335228+0.01000109j
   0.08037285+0.j        ]]
restored: [[ 0.69553616+0.j         -0.30796432+0.24210111j -0.00753329-0.048494j
  -0.16911174-0.16523753j]
 [-0.30796432-0.24210111j  0.22062831+0.j         -0.01354418+0.02409399j
   0.01736242+0.13202679j]
 [-0.00753329+0.048494j   -0.01354418-0.02409399j  0.00346268+0.j
   0.01335228-0.01000109j]
 [-0.16911174+0.16523753j  0.01736242-0.13202679j  0.01335228+0.01000109j
   0.08037285+0.j        ]]


Density matrices can be stored to files in pickle format.

In [18]:
from qulacs import QuantumState
import pickle

state = QuantumState(2)

# store
with open('state.pickle', 'wb') as f:
    pickle.dump(state, f)

# load
with open('state.pickle', 'rb') as f:
    state = pickle.load(f)


### Initialize quantum states
The following example shows functions to initialize a quantum state to a specific pure state.

In [19]:
from qulacs import DensityMatrix

n = 2
state = DensityMatrix(n)

# Initialize as |0> state.
state.set_zero_state()
print(state.get_matrix())

# Initialize as computational basis specified in binary format.
state.set_computational_basis(0b10)
print(state.get_matrix())


# Initialize as a random pure state in Haar measure with the seed given as an argument.
# If you do not give the seed, `time` function is used for seed.
# Xorshift is used for psuedo random.
state.set_Haar_random_state(0)
print(state.get_matrix())


[[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
[[ 0.06955138+0.j         -0.01203783+0.07302921j  0.10872467+0.04574116j
  -0.14160694+0.15896533j]
 [-0.01203783-0.07302921j  0.07876443+0.j          0.02921052-0.12207811j
   0.19142327+0.12117438j]
 [ 0.10872467-0.04574116j  0.02921052+0.12207811j  0.20004359+0.j
  -0.11681879+0.3416283j ]
 [-0.14160694-0.15896533j  0.19142327-0.12117438j -0.11681879-0.3416283j
   0.6516406 +0.j        ]]


### Check quantum states
The following example shows functions to check information of quantum states without changing the states.

In [20]:
from qulacs import DensityMatrix

n = 5
state = DensityMatrix(n)
state.set_Haar_random_state(0)

# Get quantum bit numbers
qubit_count = state.get_qubit_count()
print("qubit_count", qubit_count)

# Get the probability that the specified qubit will be measured as 0
prob = state.get_zero_probability(1)
print("zero_prob_1", prob)

# Get arbitrary marginal probabilities
# Argument is an array of the same length as the number of qubits
# Specify 0,1,2. 0,1 is the probability of the subscript measured at that value
# 2 means that bit is peripheralized.
# For example, calculation of the probability that the third is measured as 0 and the 0th is measured as 1:
prob = state.get_marginal_probability([1,2,2,0,2])
print("marginal_prob", prob)

# Get the entropy of the probability distribution when measured on the Z basis
ent = state.get_entropy()
print("entropy", ent)

# Get squared norm (<a|a>)
# Because the operation may not be Trace preserving, the norm of state does not necessarily to be 1.
sq_norm = state.get_squared_norm()
print("sqaured_norm", sq_norm)

# Measure and sample all the qubits on Z-basis as many times as given by the argument. 
# Returns a list of integers converted from the resulting binaries.
samples = state.sampling(10)
print("sampling", samples)

# You can supply a random seed as second argument.
# If the same seed is given, always returns the same sampling result.
samples_with_seed = state.sampling(10, 314)
print("sampling (with seed)", samples_with_seed)

# Get a character string indicating whether the state vector is on CPU or GPU
dev_type = state.get_device_name()
print("device", dev_type)


qubit_count 5
zero_prob_1 0.46010755964245986
marginal_prob 0.20030608663813237
entropy 3.108273642412474
sqaured_norm 1.0000000000000002
sampling [11, 30, 30, 30, 24, 6, 24, 3, 10, 29]
sampling (with seed) [23, 18, 28, 14, 17, 30, 9, 17, 16, 10]
device cpu


### Deformation of quantum states
The following functions modify a quantum state.
`add_state` and `multiply_coef` calculates for each element in a density matrix.

**It is fundamentally different in behavior from the operation of the same name in `QuantumState`, which corresponds to the quantum state**

- `add_state` of `QuantumState` creates superpositon, but `add_state` of `DensityMatrix` create mixed state
- The operation corresponding to `multiply_coef(z)` of `QuantumState` is `multiply_coef(abs(z)**2)` for `DensityMatrix`

Using `state.make_superposition()` `state.make_mixture()` is recommended since `add_state()` `multiply_coef()` make users confused with those generated by `QuantumState`.

In [21]:
from qulacs import DensityMatrix
state = DensityMatrix(2)
state.set_computational_basis(0)
buffer = DensityMatrix(2)
buffer.set_computational_basis(2)
print("state" , state.get_matrix())
print("buffer", buffer.get_matrix())

# Sum of quantum state (state <- state+buffer)
# Add the buffer state to the state to create a superposition state.
# The norm after the operation generally is not 1.
state.add_state(buffer)
print("added", state.get_matrix())

# Product of quantum state and complex number
# Multiplies all elements by the complex number of the argument. 
# The norm after operation generally is not 1.
coef = 3.0
state.multiply_coef(coef)
print("mul_coef", state.get_matrix())

# Normalize quantum states
# Provide the current squared norm as an argument.
squared_norm = state.get_squared_norm()
print("sq_norm", squared_norm)
state.normalize(squared_norm)
print("normalized", state.get_matrix())
print("sq_norm", state.get_squared_norm())


state [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
buffer [[0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
added [[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
mul_coef [[3.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 3.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j]]
sq_norm 6.0
normalized [[0.5+0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0.5+0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]]
sq_norm 1.0


### Operation on classical registers
`DensityMatrix` also has classical registers as integer arrays with variable length.

In [22]:
from qulacs import DensityMatrix
state = DensityMatrix(3)
position = 0
# Set the value at `position`-th register.
state.set_classical_value(position, 20)
# Get the value at `position`-th register.
obtained = state.get_classical_value(position)
print(obtained)


20


### Creating superposition states and mixture states

You can create superposition states and mixture states by using `make_superposition()` `make_mixture()` in `state` module.
These states can also be created by applying `add_state()` `multiply_coef()` to `QuantumState` `DensityMatrix`, but is deprecated due to low readability.

In [23]:
from qulacs import QuantumState, DensityMatrix
from qulacs.state import make_superposition, make_mixture
# from QuantumState |a> and |b>, create a superposition state p|a> + q|b>
a = QuantumState(2)
a.set_computational_basis(0b00)
b = QuantumState(2)
b.set_computational_basis(0b11)
p = 1 / 2
q = 1 / 2
c = make_superposition(p, a, q, b)
print(c.get_vector())
# from QuantumState |a> and DensityMatrix |b><b|, create a mixture state p|a><a| + q|b><b|
# You can also create a mixture states from two QuantumState or two DensitMatrix
a = QuantumState(2)
a.set_computational_basis(0b00)
b = DensityMatrix(2)
b.set_computational_basis(0b11)
p = 1 / 2
q = 1 / 2
c = make_mixture(p, a, q, b)
print(c.get_matrix())


[0.5+0.j 0. +0.j 0. +0.j 0.5+0.j]
[[0.5+0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0. +0.j]
 [0. +0.j 0. +0.j 0. +0.j 0.5+0.j]]


## Quantum gates

### Types of quantum gate
Quantum gates are divided into two types: special gates and general gates. In Qulacs, quantum gates are not limited to unitary operators, and the operation of updating an arbitrary quantum state, such as Instrument and CPTP-map, is also called a gate.

Special gates are those that have a pre-specified gate matrix and can only perform limited deformations on quantum gates. For example, Pauli gate, rotation Pauli gate, projection measurement, etc. are supported. The advantage of a special gate is that the update function of the quantum state is more efficient than a general gate with limited properties. Also, at the time of definition, it holds information on whether or not it is diagonalized by each qubit at the basis of some Pauli, and this information is used for circuit optimization. The disadvantage of special gates is that the possible operations on the gates are limited for the reasons mentioned above.

A gate that has an explicit operation gate matrix is called a general gate. The advantage of general gates is that you can specify the gate matrix as you like, but the disadvantage is that the updates are slower than special gates.

### Common operations for quantum gates
The gate matrix of the generated quantum gate can be obtained with the `get_matrix` function. Control qubits are not included in the gate matrix. In particular, be careful when obtain gates that do not have a gate matrix (for example, $n$-qubit Pauli rotating gates), which require a very large amount of memory and time. `print` function displays the gate information.

In [24]:
import numpy as np
from qulacs.gate import X
gate = X(2)
mat = gate.get_matrix()
print(mat)
print(gate)


[[0.+0.j 1.+0.j]
 [1.+0.j 0.+0.j]]
 *** gate info *** 
 * gate name : X
 * target    : 
 2 : commute X     
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no



Quantum gates can also be converted to/from JSON string.
Some types of gates are not supported.

In [25]:
from qulacs.gate import X
from qulacs import gate

o_gate = X(2)
print(o_gate)
gate_json = o_gate.to_json()

r_gate = gate.from_json(gate_json)
print(r_gate)


 *** gate info *** 
 * gate name : X
 * target    : 
 2 : commute X     
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

 *** gate info *** 
 * gate name : X
 * target    : 
 2 : commute X     
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no



A new instance of the same quantum gate as itself can be created with the `copy` function.

In [1]:
import numpy as np
from qulacs.gate import X

n = 2
initial_gate = X(n)
print(initial_gate)

copied_gate = initial_gate.copy()
print(copied_gate)


 *** gate info *** 
 * gate name : X
 * target    : 
 2 : commute X     
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

 *** gate info *** 
 * gate name : X
 * target    : 
 2 : commute X     
 * control   : 
 * Pauli     : yes
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no



The `update_quantum_state` function is used to change the quantum state by a quantum gate.
For example, to make the H-gate act on the 0-th qubit, write the following code.

In [2]:
import qulacs
from qulacs.gate import H

n = 2
quantum_state = qulacs.QuantumState(n)

quantum_state.set_zero_state()

hadamard_gate = H(0)

hadamard_gate.update_quantum_state(quantum_state)

print(quantum_state)


 *** Quantum State ***
 * Qubit Count : 2
 * Dimension   : 4
 * State vector : 
(0.707107,0)
(0.707107,0)
       (0,0)
       (0,0)



### Special gate
The special gates are listed below.

#### 1 qubit gate
Takes the index of the target bit as the first argument.

In [ ]:
from qulacs.gate import Identity # Identity matrix
from qulacs.gate import X, Y, Z	# Pauli
from qulacs.gate import H, S, Sdag, sqrtX, sqrtXdag, sqrtY, sqrtYdag # クリフォード
from qulacs.gate import T, Tdag # T gate
from qulacs.gate import P0, P1 # Projection to 0,1 (not normalized)
target = 3
gate = T(target)
print(gate)


 *** gate info *** 
 * gate name : T
 * target    : 
 3 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : yes
 * Parametric: no
 * Diagonal  : no



`Identity` does not update the quantum state, but when it is included into the quantum circuit, it is counted as a gate that consumes 1 step.

#### 1 qubit rotating gate
Take the index of the target bit as the first argument and the rotation angle as the second argument.

In [ ]:
import numpy as np
from qulacs.gate import RX, RY, RZ
target = 0
angle = 0.1
gate = RX(target, angle)
print(gate)
print(gate.get_matrix())


 *** gate info *** 
 * gate name : X-rotation
 * target    : 
 0 : commute X     
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

[[0.99875026+0.j         0.        +0.04997917j]
 [0.        +0.04997917j 0.99875026+0.j        ]]


The definition of rotating operation are as follows:

`RX`：
$$
R_X(\theta) = \exp(i\frac{\theta}{2} X) = 
\begin{pmatrix} 
\cos(\frac{\theta}{2}) & i\sin(\frac{\theta}{2}) \\ 
i\sin(\frac{\theta}{2}) & \cos(\frac{\theta}{2})
\end{pmatrix} 
$$ 

`RY`：
$$
R_Y(\theta) = \exp(i\frac{\theta}{2} Y) = 
\begin{pmatrix} 
\cos(\frac{\theta}{2}) & \sin(\frac{\theta}{2}) \\ 
-\sin(\frac{\theta}{2}) & \cos(\frac{\theta}{2})
\end{pmatrix} 
$$

`RZ`：
$$
R_Z(\theta) = \exp(i\frac{\theta}{2} Z) = 
\begin{pmatrix} 
e^{i\frac{\theta}{2}} & 0 \\ 
0 & e^{-i\frac{\theta}{2}}
\end{pmatrix} 
$$ 

The rotation gates used in Qulacs are basically `RX`, `RY`, and `RZ`, but there are also `RotX`, `RotY`, and `RotZ` with opposite rotation directions. The definitions of these operations are as follows:

`RotX`：$RotX(\theta) = \exp(-i\frac{\theta}{2}X)$

`RotY`：$RotY(\theta) = \exp(-i\frac{\theta}{2}Y)$

`RotZ`：$RotZ(\theta) = \exp(-i\frac{\theta}{2}Z)$

Gates with opposite rotation direction to `RotX`, `RotY` and `RotZ` are `RotInvX`, `RotInvY` and `RotInvZ`.

#### IBMQ basis gate
 IBMQ basis gate is a gate based on the virtual-Z decomposition defined by IBMQ's OpenQASM.

In [ ]:
from qulacs.gate import U1,U2,U3
print(U3(0, 0.1, 0.2, 0.3))


 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 0 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
            (0.99875,0) (-0.0477469,-0.0147699)
 (0.0489829,0.00992933)     (0.876486,0.478826)



Definitions are:

* $U_1(\lambda) = R_Z(\lambda)$
* $U_2(\phi, \lambda) = R_Z(\phi+\frac{\pi}{2}) R_X(\frac{\pi}{2}) R_Z(\lambda-\frac{\pi}{2})$
* $U_3(\theta, \phi, \lambda) = R_Z(\phi+3\pi) R_X(\pi/2) R_Z(\theta+\pi) R_X(\pi/2) R_Z(\lambda)$

U3 matches the degree of freedom of any 1-qubit unitary operation.

#### 2 qubit gate
Take the indexes of the target bit in the first and second arguments. The first argument of the CNOT gate is a control qubit. The remaining gates are symmetric operations.

In [ ]:
from qulacs.gate import CNOT, CZ, SWAP
control = 5
target = 2
target2 = 3
gate = CNOT(control, target)
print(gate)
gate = CZ(control, target)
gate = SWAP(target, target2)


 *** gate info *** 
 * gate name : CNOT
 * target    : 
 2 : commute X     
 * control   : 
 5 : value 1
 * Pauli     : no
 * Clifford  : yes
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no



#### Multi-bit Pauli operation
Multi-bit Pauli operations define a gate with arguments as a list of target qubits and a list of Pauli operators. Since the update speed of an $n$-qubit Pauli operation has the same order as the update cost of a 1-qubit Pauli operation, Pauli's tensor product should be defined as the gate in this form. In the Pauli operator specification, 1, 2, and 3 correspond to X, Y, and Z, respectively.

In [ ]:
from qulacs.gate import Pauli
target_list = [0,3,5]
pauli_index = [1,3,1] # 1:X , 2:Y, 3:Z
gate = Pauli(target_list, pauli_index) # = X_0 Z_3 X_5
print(gate)
print(gate.get_matrix())


 *** gate info *** 
 * gate name : Pauli
 * target    : 
 0 : commute X     
 3 : commute     Z 
 5 : commute X     
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

[[ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.-0.j]
 [ 0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j -1.-0.j  0.+0.j]
 [ 0.+0.j  1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 1.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j  0.+0.j -1.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]
 [ 0.+0.j  0.+0.j -1.-0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j  0.+0.j]]


#### Multi-bit Pauli rotating operation
Multi-bit Pauli rotating operation rotates multi-bit Pauli operator. The multi-bit Pauli rotation becomes heavy calculation when the gate matrix is calculated greedily, but it can be updated efficiently if it is defined in this form.

In [ ]:
from qulacs.gate import PauliRotation
target_list = [0,3,5]
pauli_index = [1,3,1] # 1:X , 2:Y, 3:Z
angle = 0.5
gate = PauliRotation(target_list, pauli_index, angle) # = exp(i angle/2 X_0 Z_3 X_5)
print(gate)
print(gate.get_matrix().shape)


 *** gate info *** 
 * gate name : Pauli-rotation
 * target    : 
 0 : commute X     
 3 : commute     Z 
 5 : commute X     
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

(8, 8)


#### Reversible circuit
Reversible circuit performs permutation operation between basis by giving a bijective function to a total number of $2^n$ index. This is equivalent to the gate matrix being a permutation matrix. Please note that it will not work properly unless the given function is bijective.

In [ ]:
from qulacs.gate import ReversibleBoolean
def upper(val, dim):
    return (val+1)%dim
target_list = [0,1]
gate = ReversibleBoolean(target_list, upper)
print(gate)
state = QuantumState(3)
state.load(np.arange(2**3))
print(state.get_vector())
gate.update_quantum_state(state)
print(state.get_vector())


 *** gate info *** 
 * gate name : ReversibleBoolean
 * target    : 
 0 : commute       
 1 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no

[0.+0.j 1.+0.j 2.+0.j 3.+0.j 4.+0.j 5.+0.j 6.+0.j 7.+0.j]
[3.+0.j 0.+0.j 1.+0.j 2.+0.j 7.+0.j 4.+0.j 5.+0.j 6.+0.j]


The above code moves the elements of the vector down one by one in the subspace of the qubit of interest (the bottom element moves to the top).
### State reflection
This gate is $(I-2\ket{a}\bra{a})$, which is defined with the quantum state $\ket{a}$ as an argument. This corresponds to the operation of reflecting based on the quantum state $\ket{a}$. This gate appears in Grover search. The number of qubits on which this gate operates must match the number of qubits in the quantum state given as an argument.

In [ ]:
from qulacs.gate import StateReflection
from qulacs import QuantumState
axis = QuantumState(2)
axis.set_Haar_random_state(0)
state = QuantumState(2)
gate = StateReflection(axis)
gate.update_quantum_state(state)
print("axis", axis.get_vector())
print("reflected", state.get_vector())
gate.update_quantum_state(state)
print("two reflection", state.get_vector())


axis [0.26990561+0.18885571j 0.42520294-0.30443016j 0.33036863-0.07179331j
 0.02371619+0.70933j   ]
reflected [-0.78296897+0.j          0.11454257-0.32493882j  0.15121954-0.16353884j
  0.28072431+0.37394642j]
two reflection [ 1.00000000e+00-2.77555756e-17j -5.55111512e-17+0.00000000e+00j
 -2.77555756e-17+2.77555756e-17j  0.00000000e+00-1.11022302e-16j]


### General gate
This is a quantum gate with a gate matrix.

### Dense matrix gate
Gate defined based on a dense matrix.

In [ ]:
from qulacs.gate import DenseMatrix

# 1-qubit gate
gate = DenseMatrix(0, [[0,1],[1,0]])
print(gate)

# 2-qubit gate
gate = DenseMatrix([0,1], [[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])
print(gate)


 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 0 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
(0,0) (1,0)
(1,0) (0,0)

 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 0 : commute       
 1 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
(1,0) (0,0) (0,0) (0,0)
(0,0) (1,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (1,0)
(0,0) (0,0) (1,0) (0,0)



### Sparse matrix gate
A gate defined based on a sparse matrix. If the elements are sparse enough, they can be updated faster than a dense matrix. Define sparse matrices using scipy's `csc_matrix`.

In [ ]:
from qulacs import QuantumState
from qulacs.gate import SparseMatrix
from scipy.sparse import csc_matrix
mat = csc_matrix((2,2))
mat[1,1] = 1
print("sparse matrix", mat)

gate = SparseMatrix([0], mat)
print(gate)

qs = QuantumState(2)
qs.load([1,2,3,4])
gate.update_quantum_state(qs)
print(qs.get_vector())


sparse matrix   (1, 1)	1.0
 *** gate info *** 
 * gate name : SparseMatrix
 * target    : 
 0 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
0 0 
0 (1,0) 


[0.+0.j 2.+0.j 0.+0.j 4.+0.j]


### Add control bit
General gates can add a control qubit using the `add_control_qubit` function. You can also specify whether the target qubit has an effect when control qubit is 0 or 1.

In [ ]:
import numpy as np
from qulacs.gate import to_matrix_gate, X

index = 0
x_gate = X(index)
x_mat_gate = to_matrix_gate(x_gate)

# Operate only when 1st-qubit is 0
control_index = 1
control_with_value = 0
x_mat_gate.add_control_qubit(control_index, control_with_value)
print(x_mat_gate)

from qulacs import QuantumState
state = QuantumState(3)
state.load(np.arange(2**3))
print(state.get_vector())

x_mat_gate.update_quantum_state(state)
print(state.get_vector())


 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 0 : commute X     
 * control   : 
 1 : value 0
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
(0,0) (1,0)
(1,0) (0,0)

[0.+0.j 1.+0.j 2.+0.j 3.+0.j 4.+0.j 5.+0.j 6.+0.j 7.+0.j]
[1.+0.j 0.+0.j 2.+0.j 3.+0.j 5.+0.j 4.+0.j 6.+0.j 7.+0.j]


## Operation to create a new gate from multiple gates
### Gate product
Combine successively the operating quantum gates to create a new single quantum gate. This reduces access to quantum states.

In [ ]:
import numpy as np
from qulacs import QuantumState
from qulacs.gate import X, RY, merge

n = 3
state = QuantumState(n)
state.set_zero_state()

index = 1
x_gate = X(index)
angle = np.pi / 4.0
ry_gate = RY(index, angle)

# Create the new gate by combining gates
# The gate in the first augement is applied first
x_and_ry_gate = merge(x_gate, ry_gate)
print(x_and_ry_gate)


 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 1 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
 (0.382683,0)   (0.92388,0)
  (0.92388,0) (-0.382683,0)



### Gate sum
You can add multiple gates to create a new gate. This is useful for making the projection of the Pauli operator $P$ onto the +1 eigenvalue space such as $(I + P) / 2$.

In [ ]:
import numpy as np
from qulacs.gate import P0,P1,add, merge, Identity, X, Z

gate00 = merge(P0(0),P0(1))
gate11 = merge(P1(0),P1(1))
# |00><00| + |11><11|
proj_00_or_11 = add(gate00, gate11)
print(proj_00_or_11)

gate_ii_zz = add(Identity(0), merge(Z(0),Z(1)))
gate_ii_xx = add(Identity(0), merge(X(0),X(1)))
proj_00_plus_11 = merge(gate_ii_zz, gate_ii_xx)
# ((|00>+|11>)(<00|+<11|))/2 = (II + ZZ)(II + XX)/4
proj_00_plus_11.multiply_scalar(0.25)
print(proj_00_plus_11)


 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 0 : commute       
 1 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
(1,0) (0,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (0,0)
(0,0) (0,0) (0,0) (1,0)

 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 0 : commute       
 1 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
(0.5,0)   (0,0)   (0,0) (0.5,0)
  (0,0)   (0,0)   (0,0)   (0,0)
  (0,0)   (0,0)   (0,0)   (0,0)
(0.5,0)   (0,0)   (0,0) (0.5,0)



### Random unitary
Use the `RandomUnitary` function to sample a random unitary matrix with the Haar measure and generate a dense matrix gate.

In [ ]:
from qulacs.gate import RandomUnitary
target_list = [2,3]
gate = RandomUnitary(target_list)
print(gate)


 *** gate info *** 
 * gate name : DenseMatrix
 * target    : 
 2 : commute       
 3 : commute       
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : no
 * Matrix
 (-0.549679,-0.199309)    (-0.41282,0.191987)  (-0.430215,-0.393607)  (-0.152234,-0.296078)
   (0.255527,0.290869)   (0.136685,-0.154401)  (0.0998426,-0.553577) (-0.700763,-0.0097331)
   (0.056699,0.690511) (-0.488394,-0.0743973)  (0.0480177,-0.260356)    (0.439362,0.113074)
 (0.156442,-0.0611172)  (-0.695389,-0.150231)    (0.177095,0.492055) (-0.433419,-0.0657552)



### Stochastic operation
Using the `Probabilistic` function, create the stochastic operation by giving multiple gate operations and probability distribution. If the sum of the given probability distributions is less than 1, `Identity` will operate with a probability less than 1.

In [ ]:
from qulacs.gate import Probabilistic, H, Z
distribution = [0.2, 0.2, 0.2]
gate_list = [H(0), Z(0), X(1)]
gate = Probabilistic(distribution, gate_list)
print(gate)

from qulacs import QuantumState
state = QuantumState(2)
for _ in range(10):
    gate.update_quantum_state(state)
    print(state.get_vector())


 *** gate info *** 
 * gate name : Generic gate
 * target    : 
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : yes

[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[ 0.+0.j -0.-0.j  1.+0.j -0.-0.j]
[0.        +0.j 0.        +0.j 0.70710678+0.j 0.70710678+0.j]
[0.        +0.j 0.        +0.j 0.70710678+0.j 0.70710678+0.j]
[ 0.        +0.j -0.        -0.j  0.70710678+0.j -0.70710678-0.j]
[ 0.70710678+0.j -0.70710678-0.j  0.        +0.j -0.        -0.j]
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[ 0.+0.j -1.-0.j  0.+0.j -0.-0.j]


`BitFlipNoise`, `DephasingNoise`, `IndependentXZNoise`, `DepolarizingNoise`, and `TwoQubitDepolarizingNoise` gates are defined as stochastic gates. `Probabilistic` instances are generated by entering the error probabilities.

In [ ]:
from qulacs.gate import BitFlipNoise, DephasingNoise, IndependentXZNoise, DepolarizingNoise, TwoQubitDepolarizingNoise
target = 0
second_target = 1
error_prob = 0.8
gate = BitFlipNoise(target, error_prob) # X: prob
gate = DephasingNoise(target, error_prob) # Z: prob
gate = IndependentXZNoise(target, error_prob) # X,Z : prob*(1-prob), Y: prob*prob
gate = DepolarizingNoise(target, error_prob) # X,Y,Z : prob/3
gate = TwoQubitDepolarizingNoise(target, second_target, error_prob) # {I,X,Y,Z} \times {I,X,Y,Z} \setminus {II} : prob/15

from qulacs import QuantumState
state = QuantumState(2)
for _ in range(10):
    gate.update_quantum_state(state)
    print(state.get_vector())


[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.-0.j 0.+1.j 0.-0.j 0.+0.j]
[0.-0.j 0.+1.j 0.-0.j 0.+0.j]
[ 0.-0.j -0.-1.j  0.-0.j -0.-0.j]
[0.+0.j 0.-0.j 0.+0.j 0.-1.j]
[0.+0.j 0.-0.j 0.+0.j 0.-1.j]
[-1.+0.j  0.+0.j -0.+0.j  0.+0.j]
[ 0.-0.j  0.+0.j  0.+0.j -0.-1.j]
[ 0.-0.j  0.+0.j  0.+0.j -0.-1.j]
[ 0.-0.j  0.+0.j  0.+0.j -0.-1.j]


### Noisy evolution
There are two gates, `NoisyEvolution` and `NoisyEvolution_fast`, that interact from the environment and are attenuated by time evolution.
They can be used by the following steps

1. set up the system to be interacted with by the environment as a Hamiltonian and specify the operator of the interaction.
2. specify the value to be evolved in time and the time to be varied microscopically.
3. solve the differential equation.
    - `NoisyEvolution` solves differential equations using Runge-Kutta method.
    - `NoisyEvolution_fast` extracts a matrix and finds it by diagonalization.

It is recommended to use `NoisyEvolution_fast` because `NoisyEvolution_fast` does not require specifying the time to make a small change and it is fast.

In [ ]:
from qulacs import QuantumState, Observable, GeneralQuantumOperator
from qulacs.gate import NoisyEvolution, NoisyEvolution_fast, H

n = 2
observable = Observable(n)
observable.add_operator(1., "X 0")

# create hamiltonian and collapse operator
hamiltonian = Observable(n)
hamiltonian.add_operator(1., "Z 0 Z 1")

decay_rate_z = 0.2
decay_rate_p = 0.6
decay_rate_m = 0.1

# interacting operator
c_ops = [GeneralQuantumOperator(n) for _ in range(3*n)]
c_ops[0].add_operator(decay_rate_z, "Z 0")
c_ops[1].add_operator(decay_rate_z, "Z 1")
c_ops[2].add_operator(decay_rate_p/2, "X 0")
c_ops[2].add_operator(decay_rate_p/2*1j, "Y 0")
c_ops[3].add_operator(decay_rate_p/2, "X 1")
c_ops[3].add_operator(decay_rate_p/2*1j, "Y 1")
c_ops[4].add_operator(decay_rate_m/2, "X 0")
c_ops[4].add_operator(-decay_rate_m/2*1j, "Y 0")
c_ops[5].add_operator(decay_rate_m/2, "X 1")
c_ops[5].add_operator(-decay_rate_m/2*1j, "Y 1")

time = 2.
gate = NoisyEvolution_fast(hamiltonian, c_ops, time)
#dt = .1
#gate = NoisyEvolution(hamiltonian, c_ops, time, dt)

exp = 0.
n_samples = 1000
state = QuantumState(n)
for k in range(n_samples):
   state.set_zero_state()
   H(0).update_quantum_state(state)
   H(1).update_quantum_state(state)
   gate.update_quantum_state(state)
   exp += observable.get_expectation_value(state) / n_samples
   print(f"[{k}] exp: {exp}")


[0] exp: -0.0006155544536970823
[1] exp: -0.0006155544536970823
[2] exp: 0.00028465213360111513
[3] exp: -0.0003309023200959672
[4] exp: -0.0009464567737930495
[5] exp: -0.0009464567737930495
[6] exp: -0.0009464567737930495
[7] exp: -0.0009464567737930495
[8] exp: -0.0009464567737930495
[9] exp: -0.0015620112274901319
[10] exp: -0.0022830973241566837
[11] exp: -0.0022830973241566837
[12] exp: -0.0022830973241566837
[13] exp: -0.002898651777853766
[14] exp: -0.002898651777853766
[15] exp: -0.0035142062315508486
[16] exp: -0.0035142062315508486
[17] exp: -0.004129760685247931
[18] exp: -0.004129760685247931
[19] exp: -0.004745315138945013
[20] exp: -0.004745315138945013
[21] exp: -0.004745315138945013
[22] exp: -0.005360869592642096
[23] exp: -0.005976424046339178
[24] exp: -0.006591978500036261
[25] exp: -0.006591978500036261
[26] exp: -0.006591978500036261
[27] exp: -0.007207532953733343
[28] exp: -0.007823087407430426
[29] exp: -0.007823087407430426
[30] exp: -0.008438641861127508
[31

### CPTP mapping
`CPTP` is created by giving a list of Claus operators that satisfy the completeness.

In [ ]:
from qulacs.gate import merge,CPTP, P0,P1

gate00 = merge(P0(0),P0(1))
gate01 = merge(P0(0),P1(1))
gate10 = merge(P1(0),P0(1))
gate11 = merge(P1(0),P1(1))

gate_list = [gate00, gate01, gate10, gate11]
gate = CPTP(gate_list)

from qulacs import QuantumState
from qulacs.gate import H,merge
state = QuantumState(2)
for _ in range(10):
    state.set_zero_state()
    merge(H(0),H(1)).update_quantum_state(state)
    gate.update_quantum_state(state)
    print(state.get_vector())


[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
[1.+0.j 0.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]
[0.+0.j 1.+0.j 0.+0.j 0.+0.j]
[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
[0.+0.j 0.+0.j 1.+0.j 0.+0.j]


The `AmplitudeDampingNoise` gate is defined as CPTP-map.

In [ ]:
from qulacs.gate import AmplitudeDampingNoise
target = 0
damping_rate = 0.1
AmplitudeDampingNoise(target, damping_rate) 
#K_0: [[1,0],[0,sqrt(1-p)]], K_1: [[0,sqrt(p)], [0,0]]


 *** gate info *** 
 * gate name : Generic gate
 * target    : 
 * control   : 
 * Pauli     : no
 * Clifford  : no
 * Gaussian  : no
 * Parametric: no
 * Diagonal  : yes

### Instrument
Instrument is an operation to get the index of the Claus operator that acts randomly in addition to the general CPTP-map operation. For example, measurement on the Z basis is equivalent to applying a CPTP-map consisting of `P0` and `P1` and knowing which one has acted. In cppsim, this is achieved in the `Instrument` function, by specifying the information of the CPTP-map and the address of the classic register in which the index of the operated Claus operator is written.

In [ ]:
from qulacs import QuantumState
from qulacs.gate import merge,Instrument, P0,P1

gate00 = merge(P0(0),P0(1))
gate01 = merge(P1(0),P0(1))
gate10 = merge(P0(0),P1(1))
gate11 = merge(P1(0),P1(1))

gate_list = [gate00, gate01, gate10, gate11]
classical_pos = 0
gate = Instrument(gate_list, classical_pos)

from qulacs import QuantumState
from qulacs.gate import H,merge
state = QuantumState(2)
for index in range(10):
    state.set_zero_state()
    merge(H(0),H(1)).update_quantum_state(state)
    gate.update_quantum_state(state)
    result = state.get_classical_value(classical_pos)
    print(index, format(result,"b").zfill(2), state.get_vector())


0 01 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
1 01 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
2 01 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
3 10 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
4 00 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
5 01 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
6 11 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
7 11 [0.+0.j 0.+0.j 0.+0.j 1.+0.j]
8 00 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]
9 10 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]


Note that `Measurement` gate is defined as Instrument.

In [ ]:
from qulacs.gate import Measurement
target = 0
classical_pos = 0
gate = Measurement(target, classical_pos)


#### ProbabilisticInstrument
In addition to the `Probabilistic` function, it is a function that can get the index of the operated gate. If the sum of the distribution is less than 1, or for some other reason no gate has acted, the number of elements in `gate_list` is saved in `classical_address`.

In [2]:
from qulacs import QuantumState
from qulacs.gate import X, Y, H, ProbabilisticInstrument
n = 2
state = QuantumState(n)
index = 0
x_gate = X(index)
y_gate = Y(index)
h_gate = H(index)
distribution = [0.25, 0.25, 0.25]
gate_list = [x_gate, y_gate, h_gate]
classical_address = 0
for _ in range(10):
  state = QuantumState(n)
  probabilistic_instrument_gate = ProbabilisticInstrument(distribution, gate_list, classical_address)
  probabilistic_instrument_gate.update_quantum_state(state)
  result = state.get_classical_value(classical_address)
  print(result, state.get_vector())


2 [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
0 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
1 [0.-0.j 0.+1.j 0.-0.j 0.+0.j]
1 [0.-0.j 0.+1.j 0.-0.j 0.+0.j]
2 [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
0 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
1 [0.-0.j 0.+1.j 0.-0.j 0.+0.j]
1 [0.-0.j 0.+1.j 0.-0.j 0.+0.j]
2 [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]
2 [0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j]


### Adaptive operation
This is a gate that uses a function that returns a Boolean value with variable-length list of classical register values as an argument, and determines whether to perform an operation according to the conditions obtained from the classical register. Conditions can be written as python functions. A python function must be a function that takes a list of type `int` as an argument and returns a bool type value.

In [ ]:
from qulacs.gate import Adaptive, X

def func(list):
    print("func is called! content is ",list)
    return list[0]==1


gate = Adaptive(X(0), func)

state = QuantumState(1)
state.set_zero_state()

# func returns False, so X does not operate
state.set_classical_value(0,0)
gate.update_quantum_state(state)
print(state.get_vector())

# func returns True, so X operates
state.set_classical_value(0,1)
gate.update_quantum_state(state)
print(state.get_vector())


func is called! content is  [0]
[1.+0.j 0.+0.j]
func is called! content is  [1]
[0.+0.j 1.+0.j]


## Operator

### Pauli operator
Observables are represented as linear combinations of Pauli operators with real coefficients. `PauliOperator` class is a class that expresses each term with the coefficient added to the $n$-qubit Pauli operator. Unlike gates, operator cannot update quantum states.

#### Create Pauli operator and obtain state

In [ ]:
from qulacs import PauliOperator
# Create Pauli Operator from pauli_string
coef = 0.1
s = "X 0 Y 1 Z 3"
pauli = PauliOperator(s, coef)
# Or, from index_list and type_list
coef =  0.1
index_list = [0, 1, 3]
type_list = "XYZ"
pauli = PauliOperator(index_list, type_list, coef)

# Added pauli symbol later
pauli.add_single_Pauli(3, 2)

# Get the subscript of each pauli symbol
index_list = pauli.get_index_list()

# Get pauli symbols (I,X,Y,Z -> 0,1,2,3)
pauli_id_list = pauli.get_pauli_id_list()

# Get pauli coefficient
coef = pauli.get_coef()

# Create a copy of pauli operator
another_pauli = pauli.copy()

s = ["I","X","Y","Z"]
pauli_str = [s[i] for i in pauli_id_list]
terms_str = [item[0]+str(item[1]) for item in zip(pauli_str,index_list)]
full_str = str(coef) + " " + " ".join(terms_str)
print(full_str)


(0.1+0j) X0 Y1 Z3 Y3


#### Expected value of Pauli operator
You can evaluate the expected value and transition moment of the Pauli operator for one state.

In [ ]:
from qulacs import PauliOperator, QuantumState

n = 5
coef = 2.0
Pauli_string = "X 0 X 1 Y 2 Z 4"
pauli = PauliOperator(Pauli_string,coef)

# Calculate expectation value <a|H|a>
state = QuantumState(n)
state.set_Haar_random_state()
value = pauli.get_expectation_value(state)
print("expect", value)

# Calculate transition moment <a|H|b>
# The first arguments comes to the bra side
bra = QuantumState(n)
bra.set_Haar_random_state()
value = pauli.get_transition_amplitude(bra, state)
print("transition", value)


expect (0.6132028798856664+0j)
transition (0.5279713648817308-0.6040063466314933j)


### General linear operators
The linear operator `GeneralQuantumOperator` is represented by a linear combination of the Pauli operators with a complex coefficient. `PauliOperator` with coefficient can be added as a term with `add_operator`.

In [ ]:
from qulacs import GeneralQuantumOperator, PauliOperator, QuantumState

n = 5
operator = GeneralQuantumOperator(n)

# Pauli operator can be added
coef = 2.0+0.5j
Pauli_string = "X 0 X 1 Y 2 Z 4"
pauli = PauliOperator(Pauli_string,coef)
operator.add_operator(pauli)
# Pauli operator can also be added directly from coefficients and strings
operator.add_operator(0.5j, "Y 1 Z 4")

# Get number of terms
term_count = operator.get_term_count()

# Get number of quantum bit
qubit_count = operator.get_qubit_count()

# Get specific terms as PauliOperator
index = 1
pauli = operator.get_term(index)


# Expected value calculation <a|H|a>
## Generally not self-adjoint, can return complex value
state = QuantumState(n)
state.set_Haar_random_state()
value = operator.get_expectation_value(state)
print("expect", value)

# Transition moment calculation <a|H|b>
# The first arguments comes to the bra side
bra = QuantumState(n)
bra.set_Haar_random_state()
value = operator.get_transition_amplitude(bra, state)
print("transition", value)


expect (-0.12123883067235244+0.004874745096201068j)
transition (-0.19076380005803-0.03521147847936834j)


### Store linear operators
`GeneralQuantumOperator` can be converted to/from JSON string.

In [ ]:
from qulacs import GeneralQuantumOperator
from qulacs import quantum_operator

o_operator = GeneralQuantumOperator(3)
o_operator.add_operator(1.0, "X 2 Y 0")
o_operator.add_operator(0.5j, "Y 1 Z 0")
operator_json = o_operator.to_json()
print("original:", o_operator)

r_operator = quantum_operator.from_json(operator_json)
print("restored:", r_operator)


original: (1,0) X 2 Y 0 + (0,0.5) Y 1 Z 0
restored: (1,0) X 2 Y 0 + (0,0.5) Y 1 Z 0


#### Generation of observables using OpenFermion
`OpenFermion` is a tool that gives Hamiltonian to be solved by chemical calculation in the form of Pauli operator. The output of this tool can be read in the form of a file or a string and can be used in the form of an operator.

In [ ]:
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_file
from qulacs.quantum_operator import create_quantum_operator_from_openfermion_text

open_fermion_text = """
(-0.8126100000000005+0j) [] +
(0.04532175+0j) [X0 Z1 X2] +
(0.04532175+0j) [X0 Z1 X2 Z3] +
(0.04532175+0j) [Y0 Z1 Y2] +
(0.04532175+0j) [Y0 Z1 Y2 Z3] +
(0.17120100000000002+0j) [Z0] +
(0.17120100000000002+0j) [Z0 Z1] +
(0.165868+0j) [Z0 Z1 Z2] +
(0.165868+0j) [Z0 Z1 Z2 Z3] +
(0.12054625+0j) [Z0 Z2] +
(0.12054625+0j) [Z0 Z2 Z3] +
(0.16862325+0j) [Z1] +
(-0.22279649999999998+0j) [Z1 Z2 Z3] +
(0.17434925+0j) [Z1 Z3] +
(-0.22279649999999998+0j) [Z2]
"""

operator = create_quantum_operator_from_openfermion_text(open_fermion_text)
print(operator.get_term_count())
print(operator.get_qubit_count())
# In the case of create_quantum_operator_from_openfermion_file, specify the path of the file where the above is written in the argument.


15
4


### Hermite operator / observable
The Hermite operator is represented by a real linear combination of the Pauli operators. Equivalent to the `GeneralQuatnumOperator` class, except that eigenvalues and expected values are guaranteed to be real.

The function to read and process from an external file is possible by replacing the `quantum_operator` with the `observable`, and using functions such as `create_observable_from_openfermion_file` `create_observable_from_openfermion_text` `create_split_observable`.

#### Separates operators into diagonal and off-diagonal terms
When reading an operator from a file, you can separate it into diagonal and off-diagonal components with the `create_split_observable` function.

In [ ]:
from qulacs.observable import create_split_observable, create_observable_from_openfermion_file

# H2.txt must be placed in openfermon format beforehand.
operator = create_observable_from_openfermion_file("./H2.txt")
diag, nondiag = create_split_observable("./H2.txt")
print(operator.get_term_count(), diag.get_term_count(), nondiag.get_term_count())
print(operator.get_qubit_count(), diag.get_qubit_count(), nondiag.get_qubit_count())


#### Compute ground state
You can compute a ground state of the operator by power method, Arnoldi method or Lanczos method. After computation, the corresponding ground state is assigned to `state` in the argument.

##### Power method

In [ ]:
from qulacs import Observable, QuantumState
from qulacs.observable import create_observable_from_openfermion_file

n = 4
operator = create_observable_from_openfermion_file("./H2.txt")
state = QuantumState(n)
state.set_Haar_random_state()
value = operator.solve_ground_state_eigenvalue_by_power_method(state, 50)
print(value)


##### Arnoldi method

In [ ]:
from qulacs import Observable, QuantumState
from qulacs.observable import create_observable_from_openfermion_file

n = 4
operator = create_observable_from_openfermion_file("./H2.txt")
state = QuantumState(n)
state.set_Haar_random_state()
value = operator.solve_ground_state_eigenvalue_by_arnoldi_method(state, 50)
print(value)


##### Lanczos method

In [ ]:
from qulacs import Observable, QuantumState
from qulacs.observable import create_observable_from_openfermion_file

n = 4
operator = create_observable_from_openfermion_file("./H2.txt")
state = QuantumState(n)
state.set_Haar_random_state()
value = operator.solve_ground_state_eigenvalue_by_lanczos_method(state, 50)
print(value)


### Apply to a quantum state
An operator can be applied to a quantum state.

In [ ]:
from qulacs import Observable, QuantumState
from qulacs.observable import create_observable_from_openfermion_file

n = 4
operator = create_observable_from_openfermion_file("./H2.txt")
state = QuantumState(n)
state.set_Haar_random_state()
result = QuantumState(n)
work_state = QuantumState(n)
operator.apply_to_state(work_state, state, result)
print(result)


## Quantum Circuits

### Structure of a quantum circuit
A quantum circuit is represented as a set of quantum gates. For example, a quantum circuit can be configured as follows.

In [ ]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import Z
n = 3
state = QuantumState(n)
state.set_zero_state()

circuit = QuantumCircuit(n)

# Add hadamard gate to quantum circuit
for i in range(n):
    circuit.add_H_gate(i)

# Create gate, which can also be added
for i in range(n):
    circuit.add_gate(Z(i))

# Operate quantum circuit to state
circuit.update_quantum_state(state)

print(state.get_vector())


[ 0.35355339+0.j -0.35355339-0.j -0.35355339-0.j  0.35355339+0.j
 -0.35355339-0.j  0.35355339+0.j  0.35355339+0.j -0.35355339-0.j]


### Further operations on a quantum circuit
You can add gates at specific positions to a quantum circuit and perform operations such as ```copy``` and ```merge_circuit```. Here is the code that summarizes these operations.

In [ ]:
from qulacs import QuantumCircuit, QuantumState
from qulacs.gate import H, Z

n = 3
state = QuantumState(n)
state.set_zero_state()

circuit = QuantumCircuit(n)

circuit.add_gate(Z(0))

# add a gate at the specified position
for i in range(n):
    circuit.add_gate(H(i), i)

circuit.remove_gate(1)

print(f"qubit count = {circuit.get_qubit_count()}")

print(f"gate count = {circuit.get_gate_count()}")

# get a deep copy of a quantum circuit.
copy_circuit = circuit.copy()

# merge the gates of the provided quantum circuit as arguments
# modifying the merged side does not affect the side that has been merged
circuit.merge_circuit(copy_circuit)

circuit.update_quantum_state(state)
print(state.get_vector())


qubit count = 3
gate count = 3
[ 0.+0.j -1.-0.j  0.+0.j -0.-0.j  0.+0.j -0.-0.j  0.+0.j -0.-0.j]


### Calculation and optimization of depth of quantum circuits
By combining quantum gates into a single quantum gate, the number of quantum gates and the time required for numerical calculations can be reduced. (Of course, if the number of target qubits increases, or if a quantum gate with a dedicated function is synthesized into a quantum gate without a dedicated function, the total calculation time may not decrease. It depends.)

The code below uses the `optimize` function to repeat the greedy synthesis of the quantum gate of the quantum circuit until the target qubit becomes three.

In [ ]:
from qulacs import QuantumCircuit
from qulacs.circuit import QuantumCircuitOptimizer
n = 5
depth = 10
circuit = QuantumCircuit(n)
for d in range(depth):
    for i in range(n):
        circuit.add_H_gate(i)

# Calculate the depth (depth=10)
print(circuit.calculate_depth())

# Optimization
opt = QuantumCircuitOptimizer()
# The maximum quantum gate size allowed to be created
max_block_size = 1
opt.optimize(circuit, max_block_size)

# Calculate the depth (depth=1へ)
print(circuit.calculate_depth())


10
1


###  Debugging quantum circuits
When print a quantum circuit, statistical information about the gates included in the quantum circuit will be displayed.

In [ ]:
from qulacs import QuantumCircuit
from qulacs.circuit import QuantumCircuitOptimizer
n = 5
depth = 10
circuit = QuantumCircuit(n)
for d in range(depth):
    for i in range(n):
        circuit.add_H_gate(i)

print(circuit)


*** Quantum Circuit Info ***
# of qubit: 5
# of step : 10
# of gate : 50
# of 1 qubit gate: 50
Clifford  : yes
Gaussian  : no




### Store QuantumCircuit
`QuantumCircuit` can be converted to/from JSON string.
If it has unsupported gates, the conversion fails with an error.

In [ ]:
from qulacs import QuantumCircuit
from qulacs import circuit

o_circuit = QuantumCircuit(3)
o_circuit.add_H_gate(0)
o_circuit.add_CNOT_gate(0, 1)
o_circuit.add_RZ_gate(2, 0.7)
circuit_json = o_circuit.to_json()
print(o_circuit)

r_circuit = circuit.from_json(circuit_json)
print(r_circuit)


*** Quantum Circuit Info ***
# of qubit: 3
# of step : 2
# of gate : 3
# of 1 qubit gate: 2
# of 2 qubit gate: 1
Clifford  : no
Gaussian  : no


*** Quantum Circuit Info ***
# of qubit: 3
# of step : 2
# of gate : 3
# of 1 qubit gate: 2
# of 2 qubit gate: 1
Clifford  : no
Gaussian  : no




Quantum circuits can be stored to files in pickle format.

In [ ]:
from qulacs import QuantumCircuit
import pickle

circuit = QuantumCircuit(3)
circuit.add_H_gate(0)
circuit.add_CNOT_gate(0, 1)
circuit.add_RZ_gate(2, 0.7)

# store
with open('circuit.pickle', 'wb') as f:
    pickle.dump(circuit, f)

# load
with open('circuit.pickle', 'rb') as f:
    circuit = pickle.load(f)


## Parametric Quantum Circuits
Defining a quantum circuit as a `ParametricQuantumCircuit` class enables some functions that are useful for optimizing quantum circuits using variational methods, in addition to the usual functions of the `QuantumCircuit` class.

### Application examples of parametric quantum circuits
Quantum gates with one rotation angle (`RX`, `RY`, `RZ`, `multi_qubit_pauli_rotation`) can be added to quantum circuits as parametric quantum gates. For a quantum gate added as a parametric gate, the number of parametric gates can be extracted after the quantum circuit is configured, and the rotation angle can be changed later.

In [ ]:
from qulacs import ParametricQuantumCircuit
from qulacs import QuantumState
import numpy as np

n = 5
depth = 10

# construct parametric quantum circuit with random rotation
circuit = ParametricQuantumCircuit(n)
for d in range(depth):
    for i in range(n):
        angle = np.random.rand()
        circuit.add_parametric_RX_gate(i,angle)
        angle = np.random.rand()
        circuit.add_parametric_RY_gate(i,angle)
        angle = np.random.rand()
        circuit.add_parametric_RZ_gate(i,angle)
    for i in range(d%2, n-1, 2):
        circuit.add_CNOT_gate(i,i+1)

# add multi-qubit Pauli rotation gate as parametric gate (X_0 Y_3 Y_1 X_4)
target = [0,3,1,4]
pauli_ids = [1,2,2,1]
angle = np.random.rand()
circuit.add_parametric_multi_Pauli_rotation_gate(target, pauli_ids, angle)

# get variable parameter count, and get current parameter
parameter_count = circuit.get_parameter_count()
param = [circuit.get_parameter(ind) for ind in range(parameter_count)]

# set 3rd parameter to 0
circuit.set_parameter(3, 0.)

# update quantum state
state = QuantumState(n)
circuit.update_quantum_state(state)

# output state and circuit info
print(state)
print(circuit)


 *** Quantum State ***
 * Qubit Count : 5
 * Dimension   : 32
 * State vector : 
   (0.0292587,0.0842756)
   (-0.0199201,0.242442)
    (0.181468,-0.104986)
  (0.0995482,-0.0233467)
     (0.16811,0.0345288)
   (0.0562338,-0.225101)
    (-0.131558,0.100174)
    (0.144257,-0.120167)
  (-0.173724,-0.0457571)
  (-0.142581,-0.0917222)
     (0.210296,0.176269)
   (0.0014233,0.0511538)
   (-0.00440611,0.26444)
   (0.0175043,0.0854573)
    (-0.197366,0.133705)
     (0.191306,0.316103)
   (-0.132828,-0.128943)
   (-0.00599114,0.03745)
  (-0.203638,-0.0249769)
   (-0.157577,0.0372862)
   (-0.132844,-0.105019)
  (-0.136556,-0.0918098)
   (-0.0584003,0.043396)
    (-0.02902,0.0680596)
   (0.0952735,-0.052631)
   (0.0738972,-0.127688)
   (0.165229,-0.0591462)
(0.0608202,-0.000749349)
  (-0.0873545,0.0887971)
    (-0.12346,0.0469901)
 (0.0344535,-0.00645451)
   (-0.216478,0.0550732)

*** Quantum Circuit Info ***
# of qubit: 5
# of step : 41
# of gate : 171
# of 1 qubit gate: 150
# of 2 qubit gate: 20

### Compute a gradient of parametric quantum circuit
You can compute a gradient of parametric circuit by `GradCalculator` or `ParametricQuantumCircuit.backprop()`.

- In `GradCalculator`, compute a gradient by calculating expected value with `CausalConeSimulator`.
- In `ParametricQuantumCircuit.backprop()`, compute a gradient by back propagation.

In [ ]:
from qulacs import ParametricQuantumCircuit, GradCalculator, Observable

n = 2
observable = Observable(n)
observable.add_operator(1.0, "X 0")
circuit = ParametricQuantumCircuit(n)

theta = [2.2, 1.4, 0.8]
circuit.add_parametric_RX_gate(0, theta[0])
circuit.add_parametric_RY_gate(0, theta[1])
circuit.add_parametric_RZ_gate(0, theta[2])

# GradCalculatorの場合
gcalc = GradCalculator()
print(gcalc.calculate_grad(circuit, observable))
# 第三引数に回転角を指定することも可能
print(gcalc.calculate_grad(circuit, observable, theta))
# Backpropを使って求めた場合
print(circuit.backprop(observable))


[(0.1329240611221506+0j), (0.06968868323709176+0j), (0.14726262077628172+0j)]
[(0.1329240611221506+0j), (0.06968868323709176+0j), (0.14726262077628172+0j)]
[0.1329240611221506, 0.06968868323709171, 0.14726262077628177]


### Store parametric quantum circuits

Parametric quantum circuits can be converted to/from JSON string.
If it has unsupported gates, the conversion fails with an error.

In [ ]:
from qulacs import ParametricQuantumCircuit
from qulacs import circuit

o_circuit = ParametricQuantumCircuit(3)
o_circuit.add_H_gate(0)
o_circuit.add_parametric_RX_gate(1, 0.3)
o_circuit.add_multi_Pauli_rotation_gate([0, 1, 2], [1, 3, 2], 1.4)
circuit_json = o_circuit.to_json()
print(o_circuit)

r_circuit = circuit.from_json(circuit_json)
print(r_circuit)


*** Quantum Circuit Info ***
# of qubit: 3
# of step : 2
# of gate : 3
# of 1 qubit gate: 2
# of 2 qubit gate: 0
# of 3 qubit gate: 1
Clifford  : no
Gaussian  : no

*** Parameter Info ***
# of parameter: 1

*** Quantum Circuit Info ***
# of qubit: 3
# of step : 2
# of gate : 3
# of 1 qubit gate: 2
# of 2 qubit gate: 0
# of 3 qubit gate: 1
Clifford  : no
Gaussian  : no

*** Parameter Info ***
# of parameter: 1



Parametric quantum circuits can be converted to files in pickle format.

In [ ]:
from qulacs import ParametricQuantumCircuit
from qulacs import circuit

circuit = ParametricQuantumCircuit(3)
circuit.add_H_gate(0)
circuit.add_parametric_RX_gate(1, 0.3)
circuit.add_multi_Pauli_rotation_gate([0, 1, 2], [1, 3, 2], 1.4)

# store
with open('circuit.pickle', 'wb') as f:
    pickle.dump(circuit, f)

# load
with open('circuit.pickle', 'rb') as f:
    circuit = pickle.load(f)


## Simulator
### `QuantumCircuitSimulator`
You can manage a circuit and quantum states together. It has a buffer for quantum state to swap the state to use.

In [ ]:
from qulacs import QuantumState, QuantumCircuit, QuantumCircuitSimulator, Observable
n = 3
state = QuantumState(n)

# 回路を作成
circuit = QuantumCircuit(n)
for i in range(n):
   circuit.add_H_gate(i)

# シミュレータクラスを作成
sim = QuantumCircuitSimulator(circuit, state)

# 基底を二進数と見た時の整数値を入れて、その状態に初期化
sim.initialize_state(0)

# ゲート数
print("gate_count: ", sim.get_gate_count())

# 実行
sim.simulate()

# 量子状態を表示
print(state)

# 期待値
observable = Observable(1)
observable.add_operator(1.0, "Z 0")
print("expectation_value: ", sim.get_expectation_value(observable))

# 量子状態を入れ替え
print("swap")
sim.swap_state_and_buffer()
# ランダムな純粋状態へ初期化
sim.initialize_random_state(seed=0)
sim.simulate()
print(state)

# 量子状態をコピー（バッファ->現状態）
sim.copy_state_from_buffer()
sim.simulate()
print(state)

# 量子状態をコピー（現状態->バッファ）
sim.copy_state_to_buffer()
sim.simulate()
print(state)


gate_count:  3
 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)

expectation_value:  0j
swap
 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
  (-0.298916,0.160953)
  (0.015405,-0.237144)
  (-0.215765,0.171064)
(-0.257959,-0.0506326)
 (-0.121612,0.0424348)
(-0.0329899,-0.400262)
  (0.327376,-0.414262)
   (0.345253,0.327824)

 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
(1,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)
(0,0)

 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)
(0.353553,0)



### `NoiseSimulator`
The `NoiseSimulator` allows you to run your circuit in a simulated noise environment.

To set up noise, you should add a gate to the circuit using `add_noise_gate()`.
The first argument specifies the gate, the second the type of noise to add (`"Depolarizing"` / `"BitFlip"` / `"Dephasing"` / `"IndependentXZ"` / `"AmplitudeDamping"`), and the third the probability of noise generation.
In the case of a two-qubit gate, you can use `"Depolarizing"` only.

In [ ]:
import random
from qulacs import QuantumState, QuantumCircuit, NoiseSimulator
from qulacs.gate import sqrtX, sqrtY, T, CNOT, CZ

n = 3
depth = 10
one_qubit_noise = ["Depolarizing", "BitFlip", "Dephasing", "IndependentXZ", "AmplitudeDamping"]
circuit = QuantumCircuit(n)

for d in range(depth):
   for i in range(n):
      r = random.randint(0, 4)
      noise_type = random.randint(0, 4)
      if r == 0:
            circuit.add_noise_gate(sqrtX(i), one_qubit_noise[noise_type], 0.01)
      elif r == 1:
            circuit.add_noise_gate(sqrtY(i), one_qubit_noise[noise_type], 0.01)
      elif r == 2:
            circuit.add_noise_gate(T(i), one_qubit_noise[noise_type], 0.01)
      elif r == 3:
            if i + 1 < n:
               circuit.add_noise_gate(CNOT(i, i+1), "Depolarizing", 0.01)
      elif r == 4:
            if i + 1 < n:
               circuit.add_noise_gate(CZ(i, i+1), "Depolarizing", 0.01)

state = QuantumState(n)
state.set_Haar_random_state()
sim = NoiseSimulator(circuit, state)
sim.execute(100)
print(state)


 *** Quantum State ***
 * Qubit Count : 3
 * Dimension   : 8
 * State vector : 
  (0.424423,-0.125395)
 (0.0193726,-0.298116)
  (-0.357495,0.227471)
   (0.0119428,0.13876)
 (-0.351555,0.0521885)
    (0.230969,0.21148)
 (0.274671,-0.0305064)
(-0.462659,-0.0337342)



### `CausalConeSimulator`
`CausalConeSimulator` allows you to extract the gates associated with a given observable by traversing the circuit in reverse.
Only the extracted gates can be applied to obtain the expected value of the physical quantity.

This allows you to find the expected value of a circuit of large size qubits, since the shallower the depth of the circuit, the fewer the gates associated with it.

In [ ]:
from qulacs import QuantumState, ParametricQuantumCircuit, CausalConeSimulator, Observable
n = 100
observable = Observable(1)
observable.add_operator(1.0, "Z 0")
circuit = ParametricQuantumCircuit(n)
for i in range(n):
   circuit.add_parametric_RX_gate(i, 1.0)
   circuit.add_parametric_RY_gate(i, 1.0)

ccs = CausalConeSimulator(circuit, observable)
print(ccs.get_expectation_value())


(0.2919265817264289+0j)
